In [0]:
from google.colab import drive
drive.mount('/content/drive')
SEED = 42

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
## main
def main():
    _process_data()
#     _load_merge_data()
#     _prepare_data()
#     _build_model()
#     _train_predict_amount()
#     _train_predict_survival()
#     _generate_submission()
    

if __name__ == "__main__":
    
    testname = 'test1'
    print(('*'* 50) +'\ntry model to test1 data\n' + ('*'*50))
    main()
    print('\n\n\nThe end.\n\n\n')
    
#     testname = 'test2'
#     print(('*'* 50) +'\ntry model to test2 data\n' + ('*'*50))
#     main()
#     print('\n\n\nThe end.\n\n\n')

**************************************************
try model to test1 data
**************************************************


NameError: ignored

In [0]:
# -*- coding: utf-8 -*-


from enum import Enum
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from datetime import datetime
import gc

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


## STEP0: do setting
class Settings(Enum):
    global train_path
    global test_path
    global submission_path
    global testname
    global codeversion
    
    codeversion = 'try 7 - delete and merge variables'
    
    path            = 'drive/My Drive/bigcontest2019/'
    train_path      = path + 'data/train/'
    test_path       = path + 'data/test/'
    submission_path = path + 'scripts/model/metrics/bjw_inference/'
        
    def __str__(self):
        return self.value
        
    
## STEP1: process data    
def fill_NA(df):
    print('\nFilling NA ...')
    
    na_ratio = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
    print('NA ratio: ')
    print(na_ratio) 
    
    for feature in df:
        if df[feature].dtype == 'object':
            df[feature] = df[feature].fillna("None")
        else:
            df[feature] = df[feature].fillna(0)
    
def encode_features(df):
    print('\nEncoding features ...')
    
    for feature in df:
        if df[feature].dtype == 'object':
            print('Encoding ', feature)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df[feature].values))
            df[feature] = lbl.transform(list(df[feature].values))
    
# def display_feature_target(df, feature):
#     fig, ax = plt.subplots()
#     ax.scatter(x = df[feature], y = df['logerror'])
#     plt.ylabel('logerror', fontsize=13)
#     plt.xlabel(feature, fontsize=13)
#     plt.show()

In [0]:
import sys
mod = sys.modules[__name__]

# function define

def week_transform(row):
    if row>0 and row<8:
        row = 1
    elif row>7 and row<15:
        row = 2
    elif row>14 and row<22:
        row = 3
    else:
        row = 4
    return row

def merge_by_char_id(df_list):
    '''
    df_list
    '''
    df = eval(df_list[0])

    for arg in df_list[1:]:
        df_arg = eval(arg)
        df = pd.concat([df, df_arg],axis=1)

    return df

def pivoting_df(df):
    
    ind = list(df.columns)[0]
    col = list(df.columns)[1]
    
    names = list(df.columns)[2:]

    for name in names:
        tmp = df.reset_index(drop=True).pivot(index = ind, columns= col,values=name).fillna(0)
        tmp.columns = [name+'_day'+str(x) for x in tmp.columns]

        setattr(mod, '{}'.format(name), tmp)

    final_df = merge_by_char_id(names)
    
    return final_df

def merge_final_df(df,*args):
    '''
    df : 기준이 되는 데이터 프레임 (char_id가 전부 있는 쪽으로!!)
    *args : 가변인자 (데이터 프레임을 원하는 만큼 집어넣으면 된다.)
    '''
    for arg in args:
        df = pd.merge(df, arg, how='left',left_index=True, right_index=True)

    return df

In [0]:
    def _process_data():
    
    print('CODE VERSION : ', codeversion)
    print('\n\nSTEP1: processing data ...')
    
    global train_merge
    global test_merge

    # load data
    print('\nLoading data ...')
    
    train_label = pd.read_csv(train_path + 'train_label.csv')
    train_activity = pd.read_csv(train_path + 'train_activity.csv')
    train_combat = pd.read_csv(train_path + 'train_combat.csv')
    train_payment = pd.read_csv(train_path + 'train_payment.csv')
    train_pledge = pd.read_csv(train_path + 'train_pledge.csv')
    train_trade = pd.read_csv(train_path + 'train_trade.csv')

    test_activity = pd.read_csv(test_path + testname + '_activity.csv')
    test_combat = pd.read_csv(test_path + testname + '_combat.csv')
    test_payment = pd.read_csv(test_path + testname + '_payment.csv')
    test_pledge = pd.read_csv(test_path + testname + '_pledge.csv')
    test_trade = pd.read_csv(test_path  + testname + '_trade.csv')

    train_activity['day'] = train_activity['day'].apply(week_transform)
    train_payment['day'] = train_payment['day'].apply(week_transform)
    train_trade['day'] = train_trade['day'].apply(week_transform)
    train_pledge['day'] = train_pledge['day'].apply(week_transform)
    train_combat['day'] = train_combat['day'].apply(week_transform)
    
    print('\nGrouping activity data ...')

    

    train_activity['game_money_change_minus'] = train_activity['game_money_change'] #game_money_change를 마이너스끼리, 플러스끼리 분리
    train_activity['game_money_change_minus'] = [0 if i > 0 else i for i in train_activity['game_money_change_minus']]
    train_activity['game_money_change'] = [0 if i < 0 else i for i in train_activity['game_money_change']]

    test_activity['game_money_change_minus'] = test_activity['game_money_change']
    test_activity['game_money_change_minus'] = [0 if i > 0 else i for i in test_activity['game_money_change_minus']]
    test_activity['game_money_change'] = [0 if i < 0 else i for i in test_activity['game_money_change']]


    activity_agg = {
         'char_id':'nunique','server':'nunique', 'playtime':'sum', 'npc_kill':'sum', #'day':'nunique',
        'solo_exp':'sum','party_exp':'sum','quest_exp':'sum','rich_monster':'sum', 
        'death':'sum', 'revive':'sum','exp_recovery':'sum','fishing':'sum','private_shop':'sum',
        'game_money_change':'sum', 'game_money_change_minus':'sum','enchant_count':'sum'}
    
    train_activity_group = train_activity.groupby(['acc_id','day'], as_index = False).agg(activity_agg).reset_index(drop=True)
    train_activity_group.columns = ['activity_{}'.format(x) for x in train_activity_group.columns()]
    train_activity_merge = pivoting_df(train_activity_group)

    test_activity_group = test_activity.groupby(['acc_id','day'], as_index = False).agg(activity_agg).reset_index(drop=True)
    test_activity_group.columns = ['activity_{}'.format(x) for x in test_activity_group.columns()]
    test_activity_merge = pivoting_df(test_activity_merge)

    acc_id_list = train_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []
    
    print('+ making day variables for train activity')

    for i in acc_id_list :
        dayunique = train_activity[train_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    train_activity_merge = train_activity_merge.merge(all_day, on = ['acc_id'], how ='left')
    
    print('+ making day variables for test activity')

    acc_id_list = test_activity['acc_id'].unique().tolist()
    login_day_max = []
    logout_day = []

    for i in acc_id_list :
        dayunique = test_activity[test_activity['acc_id'] == i]['day'].unique().tolist() # i아이디의 접속일 list화
        in_list = [0] * 28
        in_value = 0
        daylogin = 1

        out_list = [0] * 28
        out_value = 0
        day_logout = 0

        # login_day_max 변수 생성 (acc_id가 28일 중 최대 연속으로 접속한 일 수. 띄엄띄엄 접속자보다 매일 꾸준히 하는 사람 데이터를 모으고 싶었음)
        for j in range(0, len(dayunique)-1) :        
            if dayunique[j]+1 == dayunique[j+1] : # 전날 대비 연속 접속한 경우에 연속값 카운트
                daylogin += 1 
                in_list[in_value] = daylogin  # 연속값 저장
            else :
                in_value += 1 
                daylogin = 1 # 이전 접속일과 다음 접속일이 1이상 차이날 때, 연속값 초기화
        login_day_max.append(max(in_list)) # 연속 접속일 리스트 중 최대값 append

         #logout-day 변수 생성 (28일 중 후반부에만 로그인 하는 사람들과, 초반 로그인 후 접속하지 않는 유저를 구분하기 위함)
        for k in range(0, len(dayunique)-1) :
            if dayunique[k]+1 != dayunique[k+1] : #연속 접속하지 않은 경우, 다음 접속일까지 로그아웃한 일자 카운트
                day_logout = (dayunique[k+1]-dayunique[k]-1)
                out_list[out_value] = day_logout
            else :
                out_value += 1
                day_logout = 0 #접속한 경우 pass 및 연속값 초기화
        logout_day.append(max(out_list)) 

    all_day = pd.DataFrame(columns = ['acc_id', 'login_day_max','logout_day']) # 빈프레임에 day 관련 데이터 2개 추가
    all_day['acc_id'] = acc_id_list
    all_day['login_day_max'] = login_day_max
    all_day['logout_day'] = logout_day

    test_activity_merge = test_activity_merge.merge(all_day, on = ['acc_id'], how ='left')

    print('activity id unique and shape :')
    print(train_activity_merge['acc_id'].nunique(), train_activity_merge.shape)
    print(test_activity_merge['acc_id'].nunique(), test_activity_merge.shape)
    
    
    # merge payment
    print('\nGrouping payment data ...')
    
#     train_payment_merge = train_payment.groupby( 'acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)
#     test_payment_merge = test_payment.groupby('acc_id', as_index = False).agg({'day':'nunique','amount_spent':'max'}).reset_index(drop=True)

#     train_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True) #amout_spent를 일일 최고 구매량으로 변경
#     test_payment_merge.rename(columns={'amount_spent':'max_spent'}, inplace=True)



    train_payment_merge = train_payment.groupby( ['acc_id','day'], as_index = False).agg({'amount_spent':'mean'}).reset_index(drop=True)
    test_payment_merge = test_payment.groupby(['acc_id','day'], as_index = False).agg({'amount_spent':'mean'}).reset_index(drop=True)
    
    train_payment_merge.rename(columns={'amount_spent':'mean_spent'}, inplace=True) #amout_spent를 일일 평균 구매량으로 변경
    test_payment_merge.rename(columns={'amount_spent':'mean_spent'}, inplace=True)

    print('payment id unique and shape :')
    print(train_payment_merge['acc_id'].nunique(), train_payment_merge.shape)
    print(test_payment_merge['acc_id'].nunique(), test_payment_merge.shape)
    
    
    # merge combat 
    print('\nGrouping combat data ...')
    class_names = {0:'liege', 1:'knight', 2:'elf', 3:'magician', 4:'dark-elf', 5:'dragon',6:'illusionist',7:'warrior'}
    for elem in train_combat['class'].unique():train_combat[class_names[elem]] = train_combat['class'] == elem
    for elem in test_combat['class'].unique():test_combat[class_names[elem]] = test_combat['class'] == elem
    
    combat_agg = {'char_id' : 'nunique', 'server' : 'nunique', 'class' : 'nunique', 'level' : 'max', #'day':'nunique',
                  'pledge_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum', 'temp_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum','etc_cnt' : 'sum','num_opponent' : 'sum','elf' : 'sum','magician' : 'sum',
                  'warrior' : 'sum','dark-elf' : 'sum','knight' : 'sum','dragon' : 'sum','liege' : 'sum','illusionist' : 'sum'}
        
    combat_rename = {'random_attacker_cnt':'combat_random_attacker_cnt','random_defender_cnt' : 'combat_random_defender_cnt',
                     'temp_cnt' : 'combat_temp_cnt','same_pledge_cnt' : 'combat_same_pledge_cnt','etc_cnt' : 'combat_etc_cnt'}

    train_combat_merge = train_combat.groupby(['acc_id','day'], as_index = False).agg(combat_agg).reset_index(drop=True)
    test_combat_merge = test_combat.groupby(['acc_id','day'], as_index = False).agg(combat_agg).reset_index(drop=True)

    train_combat_merge.rename(columns=combat_rename, inplace=True)
    test_combat_merge.rename(columns=combat_rename, inplace=True)

    train_combat_merge = pivoting_df(train_combat_merge)
    test_combat_merge = pivoting_df(test_combat_merge)

    print('combat id unique and shape :')
    print(train_combat_merge['acc_id'].nunique(), train_combat_merge.shape)
    print(test_combat_merge['acc_id'].nunique(), test_combat_merge.shape)
    
    
    # merge combat 
    print('\nGrouping pledge data ...')
    
    pledge_agg = {'char_id':'nunique','server':'nunique','pledge_id':'nunique', 'play_char_cnt' : 'sum',
                  'combat_char_cnt' : 'sum','pledge_combat_cnt' : 'sum','random_attacker_cnt' : 'sum','random_defender_cnt' : 'sum',
                  'same_pledge_cnt' : 'sum', 'temp_cnt' : 'sum','etc_cnt' : 'sum','combat_play_time' : 'sum','non_combat_play_time' : 'sum'}
    
    train_pledge_merge = train_pledge.groupby(['acc_id','day'], as_index = False).agg(pledge_agg).reset_index(drop=True)
    test_pledge_merge = test_pledge.groupby(['acc_id','day'], as_index = False).agg(pledge_agg).reset_index(drop=True)
    
    train_pledge_merge = pivoting_df(train_pledge_merge)
    test_pledge_merge = pivoting_df(train_pledge_merge)

    print('pledge id unique and shape :')
    print(train_pledge_merge['acc_id'].nunique(), train_pledge_merge.shape)
    print(test_pledge_merge['acc_id'].nunique(), test_pledge_merge.shape)
    
    # merge trade 
    print('\nGrouping trade data ...')

    # item_type dummies             
    train_item_type_dummy = pd.get_dummies(train_trade['item_type'])
    test_item_type_dummy = pd.get_dummies(test_trade['item_type'])

    # trade_type dummies
    type_names = {0:'personal_shop', 1:'trade_shop'}
    for elem in train_trade['type'].unique():train_trade[type_names[elem]] = train_trade['type'] == elem
    for elem in test_trade['type'].unique():test_trade[type_names[elem]] = test_trade['type'] == elem

    # merge dummies
    
    train_trade = train_trade.drop(columns=['type','item_type'])
    train_trade = pd.concat([train_trade, train_item_type_dummy], axis = 1)
    
    test_trade = test_trade.drop(columns=['type','item_type'])
    test_trade = pd.concat([test_trade, test_item_type_dummy], axis = 1)

    # rename columns
    trade_rename = {'day':'trade_day','server' : 'trade_server', 'time' : 'trade_time'}
    train_trade.rename(columns=trade_rename, inplace=True)
    test_trade.rename(columns=trade_rename, inplace=True)

    #divide source and target id
    
    train_source_trade = train_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    train_source_trade['source'] = 1
    train_target_trade = train_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    train_target_trade['target'] = 1
    
    test_source_trade = test_trade.drop(columns = ['target_acc_id','source_char_id','target_char_id']).rename(columns={'source_acc_id' : 'acc_id'})
    test_source_trade['source'] = 1
    test_target_trade = test_trade.drop(columns = ['source_acc_id','source_char_id','target_char_id']).rename(columns={'target_acc_id' : 'acc_id'})
    test_target_trade['target'] = 1


    #aggregate
    trade_source_agg = { 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum', #'trade_day':'nunique',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'source' : 'sum'}

    trade_target_agg = { 'trade_time' : 'nunique', 'trade_server' : 'nunique','item_amount' : 'sum', #'trade_day':'nunique',
                        'item_price': 'sum', 'trade_shop' : 'sum', 'personal_shop' : 'sum', 'accessory' : 'sum', 'adena' : 'sum',
                        'armor' : 'sum', 'enchant_scroll' : 'sum', 'etc' : 'sum', 'spell' : 'sum', 'weapon' : 'sum', 'target' : 'sum'}

    train_source_trade = train_source_trade.groupby(['acc_id','trade_day'], as_index = False).agg(trade_source_agg).reset_index(drop=True)
    train_target_trade = train_target_trade.groupby(['acc_id','trade_day'], as_index = False).agg(trade_target_agg).reset_index(drop=True)
    
    test_source_trade = test_source_trade.groupby(['acc_id','trade_day'], as_index = False).agg(trade_source_agg).reset_index(drop=True)
    test_target_trade = test_target_trade.groupby(['acc_id','trade_day'], as_index = False).agg(trade_target_agg).reset_index(drop=True)

    train_source_trade = pivoting_df(train_source_trade)
    train_target_trade = pivoting_df(train_target_trade)

    test_source_trade = pivoting_df(test_source_trade)
    test_target_trade = pivoting_df(test_target_trade)

    print('trade id unique and shape :')
    print(train_source_trade['acc_id'].nunique(), train_source_trade.shape, train_target_trade['acc_id'].nunique(), train_target_trade.shape)
    print(test_source_trade['acc_id'].nunique(), test_source_trade.shape, test_target_trade['acc_id'].nunique(), test_target_trade.shape)
    
    
    
    #merge all data
    print('\nMerging all data ...')
    train_merge = train_label.merge(train_activity_merge, on = ['acc_id'], how ='left').merge(
    train_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    train_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    train_source_trade, on = ['acc_id'], how = 'left').merge(
    train_target_trade, on = ['acc_id'], how = 'left')
        
    test_merge = test_activity_merge.merge(
    test_combat_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_pledge_merge.drop(['day', 'char_id','server'], axis=1), on = ['acc_id'], how ='left').merge(
    test_payment_merge.drop(['day'], axis=1), on = ['acc_id'], how ='left').merge(
    test_source_trade, on = ['acc_id'], how = 'left').merge(
    test_target_trade, on = ['acc_id'], how = 'left')
    
    train_merge['attack'] = (train_merge['random_attacker_cnt']+train_merge['combat_random_attacker_cnt']) -(train_merge['random_defender_cnt'] + train_merge['combat_random_defender_cnt'])
    test_merge['attack'] = (test_merge['random_attacker_cnt']+test_merge['combat_random_attacker_cnt']) -(test_merge['random_defender_cnt'] + test_merge['combat_random_defender_cnt'])
    
    train_merge['cnt'] = train_merge['temp_cnt']+train_merge['same_pledge_cnt']+train_merge['etc_cnt'] + train_merge['combat_temp_cnt']+train_merge['combat_same_pledge_cnt']+train_merge['combat_etc_cnt']
    test_merge['cnt'] = test_merge['temp_cnt']+test_merge['same_pledge_cnt']+test_merge['etc_cnt'] + test_merge['combat_temp_cnt']+test_merge['combat_same_pledge_cnt']+test_merge['combat_etc_cnt']
    
    
    train_merge['pledge_ratio'] = train_merge['combat_char_cnt']/train_merge['play_char_cnt']
    test_merge['pledge_ratio'] = test_merge['combat_char_cnt']/test_merge['play_char_cnt']
  
    train_merge['day_spent'] = train_merge['day']*train_merge['mean_spent']
    test_merge['day_spent'] = test_merge['day']*test_merge['mean_spent']
    
    drop_merge = ['day','fishing','revive','elf','magician','warrior','dark-elf','knight','dragon','illusionist','class','random_attacker_cnt',
                  'random_defender_cnt','temp_cnt','same_pledge_cnt','etc_cnt','same_pledge_cnt','num_opponent','combat_temp_cnt', 
                  'combat_same_pledge_cnt', 'combat_etc_cnt','combat_random_defender_cnt','combat_random_attacker_cnt','playtime',
                  'combat_char_cnt','play_char_cnt','mean_spent']
    
    
    
    train_merge = train_merge.drop(drop_merge, axis = 1)
    test_merge = test_merge.drop(drop_merge, axis = 1)
    
    print('\nmerge id unique and shape :')
    print(train_merge['acc_id'].nunique(), train_merge.shape)
    print(test_merge['acc_id'].nunique(), test_merge.shape)
    
    train_merge.to_csv(submission_path + '/merge/' + 'train_merge.csv',index=False)  
    test_merge.to_csv(submission_path + '/merge/' + testname + '_merge.csv',index=False)  
    print('test and train merge data are saved on ' + submission_path + '/merge/' )
    
    
def _load_merge_data(): 

    global train_merge
    global test_merge
    print('\nSTEP1: load merged data ...')
    train_merge = pd.read_csv(submission_path + 'merge/' + 'train_merge.csv')
#     test_merge = pd.read_csv(submission_path + 'merge/' + testname + '_merge.csv')
    test_merge = pd.read_csv(submission_path + 'merge/' + 'train_merge.csv')
  
    print('\ntrain & test merge id unique and shape :')
    print('train : ', train_merge['acc_id'].nunique(), train_merge.shape)
    print('test: ', test_merge['acc_id'].nunique(), test_merge.shape)

    
    # fill NA
#     fill_NA(train_merge)
    
    # encode features
#     encode_features(train_merge)
    
### STEP1.1 : prepare train and vaild data



def _prepare_data():
    # prepare train and valid data
    print('\nPreparing train and valid data ...')

    global train_x
    global train_ya
    global train_ys
    
    global valid_x
    global valid_ya
    global valid_ys
    
    global test_x
    global test_acc_id
    
    
    drop_vars = ['acc_id','amount_spent','survival_time']
    
    train_ya = train_merge['amount_spent']
    train_ys = train_merge['survival_time']
    
    train_x = train_merge
    train_x.drop(columns=drop_vars, inplace=True)
    
    valid_x = train_x[30000:]
    valid_ya = train_ya[30000:]
    
    train_x = train_x[:30000]
    train_ya = train_ya[:30000]
    
    valid_ys = train_ys[30000:]
    train_ys = train_ys[:30000]
    
    print('train x shape: ', train_x.shape)
    print('train ya shape: ', train_ya.shape, 'train ys shape: ', train_ys.shape)

    print('valid x shape: ', valid_x.shape)
    print('valid ya shape: ', valid_ya.shape, 'valid ys shape: ', valid_ys.shape)
    
    # prepare test data
    print('\nPreparing test data ...')
    
    test_x = test_merge
    test_acc_id = test_merge['acc_id']
#     test_x.drop(columns=['acc_id'], inplace=True)
    test_x.drop(columns=['acc_id','amount_spent','survival_time'], inplace=True) #train으로 train예측
    print('test x shape: ', test_x.shape)

    
## STEP2: build model
def _build_model():
    print('\n\nSTEP2: building model ...')
    
    global xgb_params

    xgb_params = {
        'eta': 0.007,
        'max_depth': 8, 
        'subsample': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'mae',
        'lambda': 9.0,
        'alpha': 0.8,
        'colsample_bytree': 0.7,
        'silent': 1,
        'random_state' : 42
    }

    

## STEP3: train    
def _train_predict_amount():
    print('\n\nSTEP3: training amount...')
    
    global a_xgb_clf
    global a_pred
    
    ad_train = xgb.DMatrix(train_x, label=train_ya)
    ad_valid = xgb.DMatrix(valid_x, label=valid_ya)
    
    a_evals = [(ad_train, 'train'), (ad_valid, 'valid')]
    a_xgb_clf = xgb.train(xgb_params, ad_train, num_boost_round=10000, evals=a_evals, 
                        early_stopping_rounds=100, verbose_eval=200)
    

    ad_test = xgb.DMatrix(test_x)
    a_pred = a_xgb_clf.predict(ad_test)
    
def _train_predict_survival():
    print('\n\nSTEP4: training survival...')
    
    global s_xgb_clf
    global s_pred
    
    sd_train = xgb.DMatrix(train_x, label=train_ys)
    sd_valid = xgb.DMatrix(valid_x, label=valid_ys)
    
    s_evals = [(sd_train, 'train'), (sd_valid, 'valid')]
    s_xgb_clf = xgb.train(xgb_params, sd_train, num_boost_round=10000, evals=s_evals, 
                        early_stopping_rounds=100, verbose_eval=200)

    sd_test = xgb.DMatrix(test_x)
    s_pred = s_xgb_clf.predict(sd_test)
    

## STEP5: generate submission 
def _generate_submission():
    print('\n\nSTEP5: generating submission ...')
    
    test_predict = pd.DataFrame(columns = ['acc_id', 'survival_time','amount_spent'])
    
    test_predict['acc_id'] = test_acc_id
    test_predict['survival_time'] = s_pred
    test_predict['amount_spent'] = a_pred
    
    test_predict['amount_spent'] = [0 if i < 0 else i for i in test_predict['amount_spent']] # 마이너스값 0 처리 
    test_predict['survival_time'] = [1 if i < 1 else i for i in test_predict['survival_time']] # 1 미만 값 1 처리
    
    #testname에 따라 amountspent value 키우기
    if testname == 'test1' :
        test_predict['amount_spent'] = test_predict['amount_spent']*18 #best : 20
    elif testname == 'test2' :
        test_predict['amount_spent'] = test_predict['amount_spent']*5
    else :
        print('testname is wrong!')
        
    test_predict.to_csv(submission_path + testname + '_train_predict.csv',index=False)    
 